In [2]:
from constants import BIPEDAL
from PPO.PPO_agent import  ContinuousPPOAgent

import os
import gym
from PIL import Image

In [3]:
root_path = './'
weights_path = os.path.join(root_path, 'weights', BIPEDAL)
weights_1_path = os.path.join(weights_path, 'execution_1')
weights_2_path = os.path.join(weights_path, 'execution_2')

frames_path = os.path.join(root_path, 'frames')
frames_1_path = os.path.join(frames_path, 'experiment_1')
frames_2_path = os.path.join(frames_path, 'experiment_2')

In [4]:
os.makedirs(frames_path, exist_ok=True)
os.makedirs(frames_1_path, exist_ok=True)
os.makedirs(frames_2_path, exist_ok=True)

In [5]:
def test_agent(weights_path, frames_path):
    env = gym.make(BIPEDAL)
    state_shape = env.observation_space.shape
    action_space = env.action_space
    action_space_info = (action_space.shape[0], action_space.low, action_space.high)

    agent = ContinuousPPOAgent.test(weights_path, state_shape, action_space_info)

    for i in range(2):

        episode_frames = os.path.join(frames_path, f'episode_{i}')
        os.makedirs(episode_frames, exist_ok=True)

        done = False
        state = env.reset()
        total_reward = 0
        t = 0

        while not done:
            action = agent.test_step(state)
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            state = next_state

            img = env.render(mode = 'rgb_array')
            img = Image.fromarray(img)
            img.save(os.path.join(episode_frames, f'frame_{t}.jpg'))

            t += 1

        print(f'Episode {i+1}: Reward: {total_reward:.2f}')
    
    env.close()        

In [6]:
test_agent(weights_1_path, frames_1_path)
test_agent(weights_2_path, frames_2_path)

Episode 1: Reward: 316.76
Episode 2: Reward: 316.67
Episode 1: Reward: 324.36
Episode 2: Reward: 324.44


In [7]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
import os
import json
from constants import *
sns.set(style="whitegrid")

root_path = './'
results_path = os.path.join(root_path, 'results', 'health_gathering', 'execution_3')
file_path = os.path.join(results_path, 'train_results.npy')

In [13]:
with open(file_path, 'rb') as file:
    rewards = np.load(file)
    actor_loss = np.load(file)
    critic_loss = np.load(file)
    kl_divergence = np.load(file)

In [10]:
print(rewards.shape)
print(actor_loss.shape)
print(critic_loss.shape)
print(kl_divergence.shape)

(3, 3000)
(3, 1000)
(3, 1000)
(3, 1000)


In [16]:
def _plot_rewards_results(rewards):
    data = rewards[:2, :2000]
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)

    plt.figure(figsize=(7, 5))
    plt.plot(means, label='mean')
    plt.fill_between(range(means.shape[0]), means-stds, means+stds, alpha=0.3, label='mean+-std')
    plt.title(f'Last 100 episodes average reward on {VIZDOOM} using {ALGORITHM}')
    plt.xlabel('Episode')
    plt.ylabel('Episode Reward')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join('./', 'episode_rewards.png'))
    plt.close()


def _plot_train_metric_results(data, model_title_name, model_fig_name):
    data = data[:2, :800]
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)

    plt.figure(figsize=(7, 5))
    plt.plot(means, label='mean')
    plt.fill_between(range(means.shape[0]), means-stds, means+stds, alpha=0.3, label='mean+-std')
    plt.title(f'{model_title_name} evolution on {VIZDOOM} using {ALGORITHM}')
    plt.xlabel('Iteration')
    plt.ylabel(f'{model_title_name} ')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join('./', f'{model_fig_name}.png'))
    plt.close()



_plot_rewards_results(rewards)
_plot_train_metric_results(actor_loss, 'Actor Loss', 'actor_loss')
_plot_train_metric_results(critic_loss, 'Critic Loss', 'critic_loss')
_plot_train_metric_results(kl_divergence, 'KL Divergence', 'kl_divergence')

In [6]:
from vizdoom.vizdoom import DoomGame, ScreenFormat, ScreenResolution
from constants import BIPEDAL
from PPO.PPO_agent import  ContinuousPPOAgent

import os
import gym
from PIL import Image



game = DoomGame()
game.load_config('./environments/vizdoom_files/basic.cfg')
game.set_screen_resolution(ScreenResolution.RES_640X480)
game.set_screen_format(ScreenFormat.RGB24)
game.set_window_visible(True)
game.set_sound_enabled(False)
game.set_render_hud(True)
game.init()


game.new_episode()
frame = game.get_state().screen_buffer

img = Image.fromarray(frame)
img.save(os.path.join('./', f'basic_frame.jpg'))